In [4]:
from sage.graphs.graph_generators import graphs
from sage.interacts.library import interact

def kernelize(G, k):

    H = G.copy()
    

    graph_modified = True

    while graph_modified and k > 0:
        graph_modified = False 


        if len(H.vertices()) <= 1:
            break

        # 删除度数为0的顶点
        zero_degree_vertices = [v for v in H.vertices() if H.degree(v) == 0]
        if zero_degree_vertices:
            H.delete_vertices(zero_degree_vertices)
            graph_modified = True
            

        to_delete = set()
        
        # 删除度数为1的顶点及其邻居
        one_degree_vertices = [v for v in H.vertices() if H.degree(v) == 1]

        for v in one_degree_vertices:


            if v in to_delete or any(neighbor in to_delete for neighbor in H.neighbors(v)):
                continue  # 如果顶点已经在删除列表中，则跳过
            if H.degree(v) == 1:
                neighbor = H.neighbors(v)[0]
                to_delete.add(v)
                to_delete.add(neighbor)
                graph_modified = True
                k -= 1
        if len(H.vertices()) <= 1:
            break

        H.delete_vertices(list(to_delete))

        # 删除度数大于k的顶点
        high_degree_vertices = [v for v in H.vertices() if H.degree(v) > k]
        for v in high_degree_vertices:
            if H.degree(v) > 0:
                H.delete_vertex(v)
                graph_modified = True
                k -= 1 
        
        # 再次检查，确保在删除操作后图中仍有多于一个顶点
        if len(H.vertices()) <= 1:
            break
            

            
    if k < 0:
            # 如果k变成负数，返回特殊值表示无解
        return None, k
        
    if H.num_edges() > k^2:
        return None, k

        
    return H, k

@interact
def show_kernelized_graph(num_vertices=(4, 20), edge_prob=(0.1, 1.0, 0.1), k=(1, 10)):
    G = graphs.RandomGNP(num_vertices, edge_prob)
    
    if G.is_hamiltonian():
    # 获取哈密尔顿路径
        print("Hamiltonian path found.")

    else:
        print("No Hamiltonian path found.")
        
    kernel, remaining_k = kernelize(G, k)
    
    if kernel is None:
        print(f"No solution, k is reduced to {remaining_k}")
    else:
        if kernel.size() > 0:
            graphs_list = [G.plot(title="Original Graph"), kernel.plot(title=f"Kernel with remaining k = {remaining_k}")]
            graphics_array(graphs_list).show(figsize=(10, 5))
        else:
            print(f"Graph have a vertex cover of size at most {k}")

            G_plot = G.plot(title="Original Graph")
            G_plot.show()

Interactive function <function show_kernelized_graph at 0x7f0eb8dde7a0> with 3 widgets
  num_vertices: IntSlider(value=12, min=4, max=20, step=1, description='num_vertices')
  edge_prob: FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='edge_prob')
  k: IntSlider(value=5, min=1, max=10, step=1, description='k')

In [5]:
from sage.graphs.graph_generators import graphs
from sage.interacts.library import interact

def kernelize(G, k):
    H = G.copy()

    graph_modified = True
    modify_array=[]
    while graph_modified and k > 0:
        graph_modified = False

        if len(H.vertices()) <= 1:
            break

        # 删除度数为0的顶点
        zero_degree_vertices = [v for v in H.vertices() if H.degree(v) == 0]
        if zero_degree_vertices:
            H.delete_vertices(zero_degree_vertices)
            graph_modified = True
        if zero_degree_vertices:
            modify_array.append(H.plot(title=f"0-degree-del, 0-degree-list={','.join(map(str,zero_degree_vertices))}", fontsize=15))

        if len(H.vertices()) <= 1:
            break
       
 
        # 删除度数为1的顶点及其邻居
        one_degree_vertices = [v for v in H.vertices() if H.degree(v) == 1]
        print(one_degree_vertices)
        for v in one_degree_vertices:
            if H.degree(v) == 1:
                neighbor = H.neighbors(v)[0]
                
                H.delete_vertices([v, neighbor])
                modify_array.append(H.plot(title=f"node{v} 1-degree-del, neighbor={neighbor})", fontsize=15))
                graph_modified = True
                k -= 1
                if len(H.vertices()) <= 1:
                    break
        
        
        # 删除度数大于k的顶点
        high_degree_vertices = [v for v in H.vertices() if H.degree(v) > k]
        for v in high_degree_vertices:
            if H.degree(v) > 0:
                H.delete_vertex(v)
                if k>=0:
                    modify_array.append(H.plot(title=f"node{str(v)} k-degree-del k={str(k)}", fontsize=15))
                graph_modified = True
                k -= 1
            
        # 再次检查，确保在删除操作后图中仍有多于一个顶点
        if len(H.vertices()) <= 1:
            break

        if k < 0:
            # 如果k变成负数，返回特殊值表示无解
            return None, k,modify_array
    return H, k, modify_array

@interact
def show_kernelized_graph(num_vertices=(4, 20), edge_prob=(0.2, 1.0, 0.05), k=(1, 10)):
    G = graphs.RandomGNP(num_vertices, edge_prob)
    original_graph = G.copy()
    kernel, remaining_k , modify_array = kernelize(G, k)
    
    if kernel is None:
        original_graph_plot = original_graph.plot(title="Original Graph", fontsize=20)
        modify_array.insert(0, original_graph_plot)
        graphics_array(modify_array, ncols=1).show(figsize=(20, 40))
        graphics_array(modify_array, ncols=1).save(f'VC-num:{num_vertices}-k:{k}-edge_pro:{edge_prob}.png', figsize=(20, 40) )
        print(f"No solution, k is reduced to {remaining_k}")
    else:
        original_graph_plot = original_graph.plot(title="Original Graph", fontsize=20)
        kernel_plot = kernel.plot(title=f"Kernel with remaining k = {remaining_k}", fontsize=20)
        modify_array.append(kernel_plot)
        modify_array.insert(0, original_graph_plot)
        graphics_array(modify_array, ncols=1).show(figsize=(20, 40))
        graphics_array(modify_array, ncols=1).save(f'VC-num:{num_vertices}-k:{k}-edge_pro:{edge_prob}.png', figsize=(20, 40))


Interactive function <function show_kernelized_graph at 0x7f0ec5b4ab00> with 3 widgets
  num_vertices: IntSlider(value=12, min=4, max=20, step=1, description='num_vertices')
  edge_prob: FloatSlider(value=0.6000000000000001, min=0.2, max=1.0, step=0.05, description='edge_prob')
  k: IntSlider(value=5, min=1, max=10, step=1, description='k')

In [6]:
from sage.graphs.graph_generators import graphs
from sage.interacts.library import interact

def kernelize_graph(G, k):
    H = G.copy()
    graph_modified = True

    while graph_modified and k > 0:
        graph_modified = False

        # 重复检查直到图不再改变
        while True:
            initial_vertex_count = H.num_verts()

            # 删除度数为0的顶点（独立点）
            isolated_vertices = [v for v in H if H.degree(v) == 0]
            H.delete_vertices(isolated_vertices)

            # 删除孤立边（两个度数为1的顶点及其边），并减少 k
            one_degree_vertices = [v for v in H.vertices() if H.degree(v) == 1]
            for v in one_degree_vertices:
                if H.degree(v) == 1 and len(H.neighbors(v)) == 1:
                    u = H.neighbors(v)[0]
                    H.delete_edge(v, u)
                    graph_modified = True
                    k -= 1

            # 删除具有相同邻居集的顶点
            vertices_to_delete = []
            for u in H:
                for v in H:
                    if u < v and set(H.neighbors(u)) == set(H.neighbors(v)):
                        vertices_to_delete.append(v)
                        break
            H.delete_vertices(vertices_to_delete)

            if initial_vertex_count == H.num_verts():
                break  # 如果图没有变化，则退出循环

        # 检查简化后的图的顶点数是否大于 2^k
        if len(H.vertices()) > 2**k:
            return None, k  # 如果顶点数大于2^k，则原图不满足条件

    return H, k

@interact
def interact_kernelize_graph(num_vertices=(4, 20), edge_prob=(0.1, 1.0, 0.1), k=(1, 10)):
    # 生成随机图
    G = graphs.RandomGNP(num_vertices, edge_prob)
    # 应用核心化
    kernel, remaining_k = kernelize_graph(G, k)
    
    # 显示结果
    if kernel is None:
        print(f"No edge clique cover of size at most {k} exists for the original graph.")
    else:
        print(f"Remaining k after kernelization: {remaining_k}")
        graphs_list = [G.plot(title="Original Graph"), kernel.plot(title="Kernelized Graph")]
        graphics_array(graphs_list).show(figsize=(10, 5))

Interactive function <function interact_kernelize_graph at 0x7f0f3e4040d0> with 3 widgets
  num_vertices: IntSlider(value=12, min=4, max=20, step=1, description='num_vertices')
  edge_prob: FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='edge_prob')
  k: IntSlider(value=5, min=1, max=10, step=1, description='k')

In [7]:
from sage.graphs.graph_generators import graphs

def vertex_cover_approximation(G):
    """
    使用2-近似算法获取图G的顶点覆盖。
    返回顶点覆盖集合C。
    """
    C = set()
    H = G.copy()
    while H.size() > 0:
        edges = H.edges(labels=False)
        u, v = edges[0]
        C.add(u)
        C.add(v)
        H.delete_vertex(u)
        H.delete_vertex(v)
    return C

def highlight_vertices(G, vertices, title):
    """
    高亮显示指定的顶点集合，并返回图的可视化。
    """
    vertex_colors = {'red': vertices}
    return G.plot(vertex_colors=vertex_colors, title=title)

def kernelize_hamiltonian_cycle_teaching(G, k):
    """
    教学目的的核化求解哈密顿回路方法。
    展示每一步的结果。
    """
    # 步骤1: 获取顶点覆盖并展示
    C = vertex_cover_approximation(G)
    plot_vertex_cover = highlight_vertices(G, C, "Vertex Cover")
    
    # 步骤2: 构建独立集并展示
    I = set(G.vertices()) - C
    plot_independent_set = highlight_vertices(G, I, "Independent Set")
    
    # 步骤3: 展示核化后的图（示例中仅展示步骤，不进行实际核化）
    # 实际核化步骤依赖于具体问题
    
    return plot_vertex_cover, plot_independent_set

@interact
def show_kernelization_effect(num_vertices=(3, 16), prob=(0.1, 0.9, 0.1)):
    G = graphs.RandomGNP(num_vertices, prob)
    original_plot = G.plot(title="Original Graph")
    show(graphics_array([original_plot]), figsize=[10,5])
    plots = kernelize_hamiltonian_cycle_teaching(G, k=2)
    graphics_array(plots).show(figsize=[15, 5])

Interactive function <function show_kernelization_effect at 0x7f0eb8553f40> with 2 widgets
  num_vertices: IntSlider(value=9, min=3, max=16, step=1, description='num_vertices')
  prob: FloatSlider(value=0.5, min=0.1, max=0.9, step=0.1, description='prob')